# Content-Based Recommendation

### 1) TF-IDF Matrisinin Oluşturulması

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv("/movies_metadata.csv", low_memory=False)
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
df.shape

(45466, 24)

In [4]:
df["overview"].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [5]:
df[df["overview"].isnull()]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
32,False,NaN,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 12, ...",NaN,78802,tt0114952,fr,"Guillaumet, les ailes du courage",NaN,...,1996-09-18,0.0,50.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Wings of Courage,False,6.8,4.0
300,False,NaN,22000000,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",NaN,161495,tt0114296,sv,Roommates,NaN,...,1995-03-01,12400000.0,108.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Roommates,False,6.4,7.0
634,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,287305,tt0117312,de,Peanuts – Die Bank zahlt alles,NaN,...,1996-03-21,0.0,NaN,[],Released,NaN,Peanuts – Die Bank zahlt alles,False,4.0,1.0
635,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,339428,tt0116485,de,Happy Weekend,NaN,...,1996-03-14,65335.0,NaN,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,NaN,Happy Weekend,False,0.0,0.0
641,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,10801,tt0117788,de,Das Superweib,NaN,...,1996-03-06,0.0,86.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,NaN,The Superwife,False,5.3,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45342,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,199887,tt1771636,en,Over/Under,NaN,...,2013-01-04,0.0,87.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Over/Under,False,4.0,2.0
45377,False,NaN,0,"[{'id': 12, 'name': 'Adventure'}]",NaN,317389,tt0070695,es,Simbad e il califfo di Bagdad,NaN,...,1973-07-22,0.0,NaN,"[{'iso_639_1': 'it', 'name': 'Italiano'}]",Released,NaN,Simbad e il califfo di Bagdad,False,0.0,0.0
45398,False,NaN,1254040,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",http://lmtr.fi/,468707,tt5742932,fi,Lauri Mäntyvaaran tuuheet ripset,NaN,...,2017-07-28,0.0,90.0,"[{'iso_639_1': 'fi', 'name': 'suomi'}]",Released,NaN,Thick Lashes of Lauri Mäntyvaara,False,8.0,1.0
45399,False,NaN,750000,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",NaN,280422,tt3805180,ru,Все и сразу,NaN,...,2014-06-05,3.0,0.0,"[{'iso_639_1': 'ru', 'name': 'Pусский'}]",Released,NaN,All at Once,False,6.0,4.0


In [9]:
df["overview"] = df["overview"].fillna("")
df[df["overview"].isnull()]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count


In [13]:
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(df["overview"])

print(tfidf_matrix)

  (0, 17764)	0.1348314953863925
  (0, 4388)	0.14748820342184052
  (0, 38030)	0.10142919482788752
  (0, 21887)	0.10438761058719499
  (0, 19641)	0.13281884272823927
  (0, 48558)	0.10339358185033236
  (0, 59519)	0.1300801610445509
  (0, 12490)	0.12544427954397822
  (0, 51108)	0.1343481728311918
  (0, 29238)	0.10093917370354447
  (0, 50914)	0.09190797940163037
  (0, 39423)	0.11907123344715954
  (0, 1847)	0.140911774178889
  (0, 58571)	0.11355918868736861
  (0, 38693)	0.2062792468281062
  (0, 9874)	0.502803868613561
  (0, 9087)	0.10635375129287979
  (0, 7491)	0.12380553184830105
  (0, 56872)	0.11124851086523602
  (0, 28729)	0.13311522181618415
  (0, 39012)	0.0871868917895906
  (0, 67874)	0.14878284660693247
  (0, 3159)	0.4117836571172595
  (0, 73468)	0.4809827114790238
  (0, 38088)	0.10739705953465474
  :	:
  (45464, 26957)	0.0735096263170162
  (45464, 18919)	0.09271509240923416
  (45464, 18119)	0.07466631763708825
  (45464, 39012)	0.0682961777913538
  (45465, 16520)	0.32373307886945113
  (

In [14]:
tfidf_matrix.shape

(45466, 75827)

In [15]:
tfidf.get_feature_names_out() # tf-idf modelinin oluşturduğu sözcükleri gösterir
tfidf_matrix.toarray() # tf-idf matrisini diziye çevirir

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### 2) Cosine Similarity

In [16]:
cosine_sim = cosine_similarity(tfidf_matrix,
                               tfidf_matrix)

# Bütün olası döküman çiftleri için cosine similarity hesaplar.

cosine_sim.shape

(45466, 45466)

### 3) Benzerliklere Göre Önerilerin Yapılması

In [17]:
indices = pd.Series(df.index, index=df['title']) # df'in index'ini title'a çevirir

indices.index.value_counts() # title'ın kaç kere tekrar ettiğini gösterir

title
Cinderella              11
Hamlet                   9
Alice in Wonderland      9
Beauty and the Beast     8
Les Misérables           8
                        ..
Cluny Brown              1
Babies                   1
The Green Room           1
Captain Conan            1
Queerama                 1
Name: count, Length: 42277, dtype: int64

In [19]:
indices = indices[~indices.index.duplicated(keep='last')] # Çoklama Problemini ortadan kaldırır
movie_index = indices["Sherlock Holmes"] # örnek bir filmin indeksi bir değişkene atandı

cosine_sim[movie_index] # Sherlock Holmes ile diğer tüm filmler arasındaki benzerlikleri gösterir

array([0.        , 0.00392837, 0.00476764, ..., 0.        , 0.0067919 ,
       0.        ])

In [20]:
similarity_scores = pd.DataFrame(cosine_sim[movie_index],
                                 columns=["score"]) # üstteki cosine_sim matrisini dataframe'e çevirir

movie_indices = similarity_scores.sort_values("score", ascending=False)[1:11].index # 0. index'i almamızın sebebi bu indexte filmin kendisi var.

df['title'].iloc[movie_indices] # iloc: yukarıdaki movie_indicies ' e göre seçim yapar.

34737    Приключения Шерлока Холмса и доктора Ватсона: ...
14821                                    The Royal Scandal
34750    The Adventures of Sherlock Holmes and Doctor W...
9743                           The Seven-Per-Cent Solution
4434                                        Without a Clue
29706                       How Sherlock Changed the World
18258                   Sherlock Holmes: A Game of Shadows
24665     The Sign of Four: Sherlock Holmes' Greatest Case
6432                   The Private Life of Sherlock Holmes
29154                          Sherlock Holmes in New York
Name: title, dtype: object

### 4) Fonksiyonu

In [23]:
def content_based_recommender(title, cosine_sim, dataframe):
    indices = pd.Series(dataframe.index, index=dataframe['title'])
    indices = indices[~indices.index.duplicated(keep='last')]
    movie_index = indices[title]
    similarity_scores = pd.DataFrame(cosine_sim[movie_index], columns=["score"])
    movie_indices = similarity_scores.sort_values("score", ascending=False)[1:11].index
    return dataframe['title'].iloc[movie_indices]

def calculate_cosine_sim(dataframe):
    tfidf = TfidfVectorizer(stop_words='english')
    dataframe['overview'] = dataframe['overview'].fillna('')
    tfidf_matrix = tfidf.fit_transform(dataframe['overview'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cosine_sim

In [24]:
content_based_recommender("Sherlock Holmes", cosine_sim, df)

34737    Приключения Шерлока Холмса и доктора Ватсона: ...
14821                                    The Royal Scandal
34750    The Adventures of Sherlock Holmes and Doctor W...
9743                           The Seven-Per-Cent Solution
4434                                        Without a Clue
29706                       How Sherlock Changed the World
18258                   Sherlock Holmes: A Game of Shadows
24665     The Sign of Four: Sherlock Holmes' Greatest Case
6432                   The Private Life of Sherlock Holmes
29154                          Sherlock Holmes in New York
Name: title, dtype: object

In [25]:
content_based_recommender("The Matrix", cosine_sim, df)

44161                        A Detective Story
44167                              Kid's Story
44163                             World Record
33854                                Algorithm
167                                    Hackers
20707    Underground: The Julian Assange Story
6515                                  Commando
24202                                 Who Am I
22085                           Berlin Express
9159                                  Takedown
Name: title, dtype: object